In [1]:
import sys
import requests
from bs4 import BeautifulSoup
from datetime import date, datetime, timedelta
import pandas as pd
from urllib import parse
import os

In [2]:
from konlpy.tag import Mecab, Komoran

In [3]:
tokenizer = Komoran()

In [4]:
stop_words = {'신종'}

In [5]:
'신종' in stop_words

True

In [6]:
total = []
for roots, dirs, files in os.walk("./20201108"):
    for idx,file in enumerate(files):
        if idx % 100 ==0 : print(idx)
        with open (roots + "/" + file, "r", encoding='utf-8') as f:
            for text in f:
                tmp = []
                for word, morpheme in tokenizer.pos(text.strip()):
                    if morpheme in ['NNG', 'NNP', 'NNB', "NNM"] and len(word) > 1 :
                        #print(word, end=", ")
                        if word not in stop_words: tmp.append(word)
        total.append(tmp)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900


In [7]:
len(total)

1970

In [1]:
from gensim.models import word2vec

C:\ProgramData\Anaconda3\envs\pydata\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
!pip install python-Levenshtein

  Running setup.py clean for python-Levenshtein
Failed to build python-Levenshtein
    Running setup.py install for python-Levenshtein: started
    Running setup.py install for python-Levenshtein: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\ProgramData\Anaconda3\envs\pydata\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Playdata\\AppData\\Local\\Temp\\pip-install-vj60cx1q\\python-levenshtein_a5b4ee3a4e154d4c8736be13fb5592fd\\setup.py'"'"'; __file__='"'"'C:\\Users\\Playdata\\AppData\\Local\\Temp\\pip-install-vj60cx1q\\python-levenshtein_a5b4ee3a4e154d4c8736be13fb5592fd\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Playdata\AppData\Local\Temp\pip-wheel-p373b1cb'
       cwd: C:\Users\Playdata\AppData\Local\Temp\pip-install-vj60cx1q\python-levenshtein_a5b4ee3a4e154d4c8736be13fb5592fd\
  Complete output (27 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.7
  creating build\lib.win-amd64-3.7\Levenshtein
  copyi

#### min_count 등장횟수가 특정 값 이해는 제외
#### sample : 빈번하게 등장하는 단어에 대한 다운 샘플링 : Google 문서는 .00001에서 .001 사이의 값을 권장한다. 
#### 여기에서는 0.001에 가까운 값이 최종 모델의 정확도를 높이는 것으로 보여진다.
#### size : 많은 feature를 사용한다고 항상 좋은 것은 아니지만 대체적으로 좀 더 나은 모델이 된다.
#### window : 학습 알고리즘이 고려해야 하는 컨텍스트의 단어 수. hierarchical softmax 를 위해 좀 더 큰 수가 좋지만 10 정도가 적당하다.

#### Hierarchical Softmax
* 계산량이 많은 Softmax function 대신 보다 빠르게 계산가능한 multinomial distribution function을 사용하는 테크닉이다. 이 방법에서는 각 단어들을 leaves로 가지는 binary tree를 하나 만들어놓은 다음(complete 할 필요는 없지만, full 할 필요는 있을 것 같다), 해당하는 단어의 확률을 계산할 때 root에서부터 해당 leaf로 가는 path에 따라서 확률을 곱해나가는 식으로 해당 단어가 나올 최종적인 확률을 계산한다.

In [9]:
model = word2vec.Word2Vec(total, workers = 4, vector_size=300, min_count=40, 
                 window=10, sample=0.001) #workers : 몇개가 일할지, window : 구글에서 10정도가 좋다고 함(구글에서 만든 모델임)

In [10]:
#코싸인 유사도가 같은 것들을 보여준 것
model.wv.most_similar("바이든")

[('행정부', 0.9322667121887207),
 ('웨이브', 0.8937715888023376),
 ('당선', 0.8926633596420288),
 ('공화당', 0.8884977102279663),
 ('선인', 0.8835152387619019),
 ('트럼프', 0.8809217214584351),
 ('블루', 0.8799217343330383),
 ('민주당', 0.8739665746688843),
 ('하원', 0.8706897497177124),
 ('수혜', 0.8650088310241699)]

In [ ]:
#벡터보는 법
model.wv['백신']

In [ ]:
model.wv['백신'].shape #300차원 -> 우리는 아무리 봐도 그 의미를 알 수 없음